## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sarkand,KZ,45.4100,79.9186,28.74,96,100,3.31,light snow
1,1,Ornskoldsvik,SE,63.2909,18.7153,28.22,68,100,18.41,overcast clouds
2,2,Yellowknife,CA,62.4560,-114.3525,-14.71,76,40,8.05,scattered clouds
3,3,Bethel,US,41.3712,-73.4140,45.05,66,75,5.75,broken clouds
4,4,Upernavik,GL,72.7868,-56.1549,27.82,85,100,3.04,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Bonthe,SL,7.5264,-12.5050,77.22,81,7,3.11,clear sky
7,7,New Norfolk,AU,-42.7826,147.0587,77.67,25,73,1.99,broken clouds
11,11,Bintulu,MY,3.1667,113.0333,82.81,78,40,5.75,scattered clouds
12,12,Rikitea,PF,-23.1203,-134.9692,77.68,78,100,13.98,overcast clouds
13,13,Camocim,BR,-2.9022,-40.8411,76.15,89,23,7.83,few clouds
17,17,Lethem,GY,3.3803,-59.7968,77.88,76,82,12.10,broken clouds
23,23,Avarua,CK,-21.2078,-159.7750,78.85,83,100,8.05,overcast clouds
24,24,Souillac,MU,-20.5167,57.5167,84.36,83,40,12.66,scattered clouds
25,25,Road Town,VG,18.4167,-64.6167,77.00,71,3,21.43,clear sky
36,36,Carnarvon,AU,-24.8667,113.6333,84.27,65,0,12.66,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             159
City                   159
Country                159
Lat                    159
Lng                    159
Max Temp               159
Humidity               159
Cloudiness             159
Wind Speed             159
Current Description    159
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

159


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Bonthe,SL,7.5264,-12.5050,77.22,81,7,3.11,clear sky
7,7,New Norfolk,AU,-42.7826,147.0587,77.67,25,73,1.99,broken clouds
11,11,Bintulu,MY,3.1667,113.0333,82.81,78,40,5.75,scattered clouds
12,12,Rikitea,PF,-23.1203,-134.9692,77.68,78,100,13.98,overcast clouds
13,13,Camocim,BR,-2.9022,-40.8411,76.15,89,23,7.83,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Bonthe,SL,77.22,clear sky,7.5264,-12.5050,
7,New Norfolk,AU,77.67,broken clouds,-42.7826,147.0587,
11,Bintulu,MY,82.81,scattered clouds,3.1667,113.0333,
12,Rikitea,PF,77.68,overcast clouds,-23.1203,-134.9692,
13,Camocim,BR,76.15,few clouds,-2.9022,-40.8411,
17,Lethem,GY,77.88,broken clouds,3.3803,-59.7968,
23,Avarua,CK,78.85,overcast clouds,-21.2078,-159.7750,
24,Souillac,MU,84.36,scattered clouds,-20.5167,57.5167,
25,Road Town,VG,77.00,clear sky,18.4167,-64.6167,
36,Carnarvon,AU,84.27,clear sky,-24.8667,113.6333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Bonthe,SL,77.22,clear sky,7.5264,-12.5050,Bonthe Holiday Village
7,New Norfolk,AU,77.67,broken clouds,-42.7826,147.0587,The Shingles Riverside Cottages
11,Bintulu,MY,82.81,scattered clouds,3.1667,113.0333,900 Boutique INN
12,Rikitea,PF,77.68,overcast clouds,-23.1203,-134.9692,People ThankYou
13,Camocim,BR,76.15,few clouds,-2.9022,-40.8411,Ilha Praia Hotel
17,Lethem,GY,77.88,broken clouds,3.3803,-59.7968,Savannah Inn
23,Avarua,CK,78.85,overcast clouds,-21.2078,-159.7750,Paradise Inn
24,Souillac,MU,84.36,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
25,Road Town,VG,77.00,clear sky,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
36,Carnarvon,AU,84.27,clear sky,-24.8667,113.6333,Hospitality Carnarvon


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))